In [21]:
import csv
import os
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras
import pickle
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Convolution2D, MaxPooling2D, Flatten
from keras.utils import np_utils

In [22]:
classes = ['Acer_Capillipes', 'Acer_Circinatum', 'Acer_Mono', 'Acer_Opalus', 'Acer_Palmatum', 'Acer_Pictum', 'Acer_Platanoids', 'Acer_Rubrum', 'Acer_Rufinerve', 'Acer_Saccharinum', 'Alnus_Cordata', 'Alnus_Maximowiczii', 'Alnus_Rubra', 'Alnus_Sieboldiana', 'Alnus_Viridis', 'Arundinaria_Simonii', 'Betula_Austrosinensis', 'Betula_Pendula', 'Callicarpa_Bodinieri', 'Castanea_Sativa', 'Celtis_Koraiensis', 'Cercis_Siliquastrum', 'Cornus_Chinensis', 'Cornus_Controversa', 'Cornus_Macrophylla', 'Cotinus_Coggygria', 'Crataegus_Monogyna', 'Cytisus_Battandieri', 'Eucalyptus_Glaucescens', 'Eucalyptus_Neglecta', 'Eucalyptus_Urnigera', 'Fagus_Sylvatica', 'Ginkgo_Biloba', 'Ilex_Aquifolium', 'Ilex_Cornuta', 'Liquidambar_Styraciflua', 'Liriodendron_Tulipifera', 'Lithocarpus_Cleistocarpus', 'Lithocarpus_Edulis', 'Magnolia_Heptapeta', 'Magnolia_Salicifolia', 'Morus_Nigra', 'Olea_Europaea', 'Phildelphus', 'Populus_Adenopoda', 'Populus_Grandidentata', 'Populus_Nigra', 'Prunus_Avium', 'Prunus_X_Shmittii', 'Pterocarya_Stenoptera', 'Quercus_Afares', 'Quercus_Agrifolia', 'Quercus_Alnifolia', 'Quercus_Brantii', 'Quercus_Canariensis', 'Quercus_Castaneifolia', 'Quercus_Cerris', 'Quercus_Chrysolepis', 'Quercus_Coccifera', 'Quercus_Coccinea', 'Quercus_Crassifolia', 'Quercus_Crassipes', 'Quercus_Dolicholepis', 'Quercus_Ellipsoidalis', 'Quercus_Greggii', 'Quercus_Hartwissiana', 'Quercus_Ilex', 'Quercus_Imbricaria', 'Quercus_Infectoria_sub', 'Quercus_Kewensis', 'Quercus_Nigra', 'Quercus_Palustris', 'Quercus_Phellos', 'Quercus_Phillyraeoides', 'Quercus_Pontica', 'Quercus_Pubescens', 'Quercus_Pyrenaica', 'Quercus_Rhysophylla', 'Quercus_Rubra', 'Quercus_Semecarpifolia', 'Quercus_Shumardii', 'Quercus_Suber', 'Quercus_Texana', 'Quercus_Trojana', 'Quercus_Variabilis', 'Quercus_Vulcanica', 'Quercus_x_Hispanica', 'Quercus_x_Turneri', 'Rhododendron_x_Russellianum', 'Salix_Fragilis', 'Salix_Intergra', 'Sorbus_Aria', 'Tilia_Oliveri', 'Tilia_Platyphyllos', 'Tilia_Tomentosa', 'Ulmus_Bergmanniana', 'Viburnum_Tinus', 'Viburnum_x_Rhytidophylloides', 'Zelkova_Serrata']
num_classes = len(classes)
num_labels = 990
image_size = 500
print(len(classes))

99


In [23]:
train = pd.read_csv('train.csv')
parent_data = train.copy()
test = pd.read_csv('test.csv')
print(train.shape)

(990, 194)


In [24]:
def dense_to_one_hot(train_list):
    """Convert class labels from scalars to one-hot vectors."""
    labels_one_hot = np.zeros((len(train_list), num_classes))
    for i in xrange(len(train_list)):
    	labels_one_hot[i][train_list[i]] = 1 
    return labels_one_hot


In [25]:
train_id = train.pop('id')
train_labels = train.pop('species')
train_labels = LabelEncoder().fit(train_labels).transform(train_labels)
print train_labels.shape

(990,)


In [26]:
print train_labels.shape
print train_labels[0]
train_labels_one_hot = dense_to_one_hot(train_labels)
print (train_labels_one_hot[0])

print (train_labels_one_hot.shape)
train_data = StandardScaler().fit(train).transform(train)
print train_data.shape


(990,)
3
[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.]
(990, 99)
(990, 192)


In [27]:
test = pd.read_csv('test.csv')
test_index = test.pop('id')
test_data = StandardScaler().fit(test).transform(test)
print test_data.shape

(594, 192)


In [34]:
folder = 'images_pre'
image_files = os.listdir('images_pre')
image_width = 500
image_height = 500
dataset = np.zeros(shape=(len(image_files), image_width, image_height),
                       dtype=np.float32)


In [37]:
for image_index, image in enumerate(image_files):
  image_file = os.path.join(folder, image)
  image_data = ndi.imread(image_file, flatten=True).astype(float)
  dataset[image_index-1,:image_data.shape[0],:image_data.shape[1]] = image_data

In [40]:
test_image_data = np.zeros(shape = (594,500,500,1),dtype=float)
train_image_data = np.zeros(shape = (990,500,500,1),dtype=float)
print test_index.shape

for i in xrange(990):
    train_image_data[i,:,:,0] = dataset[train_id[i]-1,:,:]

for i in xrange(594):
    test_image_data[i,:,:,0] = dataset[test_index[i]-1,:,:]

print train_image_data.shape
del dataset

(594,)
(990, 500, 500, 1)


In [41]:
sss = StratifiedShuffleSplit(train_labels,test_size=0.1, random_state=23)

for train_index, test_index in sss:
    train_data_2, valid_data = train_data[train_index], train_data[test_index]
    train_image_data_2, valid_image_data = train_image_data[train_index], train_image_data[test_index]
    train_labels_2, valid_labels = train_labels[train_index], train_labels[test_index]
    train_labels_one_hot_2, valid_labels_one_hot = train_labels_one_hot[train_index], train_labels_one_hot[test_index]
print valid_data.shape, valid_labels_one_hot.shape, valid_image_data.shape

(99, 192) (99, 99) (99, 500, 500, 1)


In [42]:
del train_data, train_labels_one_hot, train_image_data

In [172]:
input_shape = (1,image_size,image_size)
kernel_size = (5, 5)
pool_size = (2, 2)
feature_depth = 8

In [173]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


batch_size = 16
patch_size = 8
depth = 400
num_hidden = 64
num_channels = 1
num_labels = 99
graph = tf.Graph()

In [ ]:
with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_image_data,dtype = tf.float32)
  tf_test_dataset = tf.constant(test_image_data, dtype = tf.float32)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 4, 4, 1], strides=[1, 2, 2, 1], padding='SAME')
    shape = pool.get_shape().as_list()
    print shape
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = loss + 0.01 * tf.nn.l2_loss(layer3_weights)+ 0.01 * tf.nn.l2_loss(layer4_weights)+0.01 * tf.nn.l2_loss(layer1_weights)+0.01 * tf.nn.l2_loss(layer2_weights)
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 64, 64, 400]
[98, 64, 64, 400]
[594, 64, 64, 400]


In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  #train_writer = tf.train.SummaryWriter('/tmp/tf_log', sess.graph)
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels_one_hot_2.shape[0] - batch_size)
    batch_data = train_image_data_2[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels_one_hot_2[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels_one_hot))
    